<a href="https://colab.research.google.com/github/fazalpge/product_forecasting/blob/main/Prophet_Complete_Example_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [ ]:
class ProphetModel(BaseEstimator, TransformerMixin):
    def __init__(self, periods=30, yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False, holidays=None):
        self.periods = periods
        self.yearly_seasonality = yearly_seasonality
        self.weekly_seasonality = weekly_seasonality
        self.daily_seasonality = daily_seasonality
        self.holidays = holidays
        self.models = {}

    def fit(self, X, y=None):
        # Extract unique combinations of 'Location' and 'ItemName'
        unique_combinations = X[['Location', 'ItemName']].drop_duplicates()
        for _, row in unique_combinations.iterrows():
            location = row['Location']
            item_name = row['ItemName']
            # Filter data for the current combination of 'Location' and 'ItemName'
            sub_df = X[(X['Location'] == location) & (X['ItemName'] == item_name)].copy()
            # Rename columns for Prophet
            sub_df = sub_df.rename(columns={'TransactionDate': 'ds', 'SaleAmount': 'y'})
            # Ensure the 'ds' column is in datetime format
            sub_df['ds'] = pd.to_datetime(sub_df['ds'], format='%Y-%m-%d')
            # Check if there are at least 2 non-NaN rows
            if sub_df.dropna().shape[0] >= 2:
                # Initialize and fit Prophet model
                model = Prophet(
                    yearly_seasonality=self.yearly_seasonality,
                    weekly_seasonality=self.weekly_seasonality,
                    daily_seasonality=self.daily_seasonality,
                    holidays=self.holidays
                )
                model.fit(sub_df)
                self.models[(location, item_name)] = model
            else:
                print(f"Skipping combination Location: {location}, ItemName: {item_name} due to insufficient data")
        return self

    def transform(self, X):
        forecast_df = pd.DataFrame()
        # Extract unique combinations of 'Location' and 'ItemName'
        unique_combinations = X[['Location', 'ItemName']].drop_duplicates()
        for _, row in unique_combinations.iterrows():
            location = row['Location']
            item_name = row['ItemName']
            if (location, item_name) in self.models:
                model = self.models[(location, item_name)]
                # Create future dataframe for forecasting
                future = model.make_future_dataframe(periods=self.periods)
                # Generate forecast
                forecast = model.predict(future)
                # Add 'Location' and 'ItemName' to forecast
                forecast['Location'] = location
                forecast['ItemName'] = item_name
                # Concatenate forecasts for all combinations
                forecast_df = pd.concat([forecast_df, forecast], ignore_index=True)
        return forecast_df

In [ ]:
# Load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/fazalpge/product_forecasting/main/Inventory%20Data_fazal.csv')


In [ ]:
df.isnull().sum()

TransactionDate            0
Location                   0
ItemName                   0
Cost                       0
SaleQty                    0
SaleBonusQty               0
SaleTotalQty               0
MRP                        0
SaleAmount                 0
PerInvoiceAverageAmount    0
dtype: int64

In [ ]:
# Ensure the date column is in datetime format
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

<ipython-input-32-af55811504b3>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])


In [ ]:
# Define holidays (example, you should replace this with actual holiday dates)
holidays = pd.DataFrame({
    'holiday': 'example_holiday',
    'ds': pd.to_datetime(['2023-01-01', '2023-12-25'], format='%Y-%m-%d'),
    'lower_window': 0,
    'upper_window': 1,
})

In [ ]:
# Create a pipeline
pipeline_prophet = Pipeline([
    ('prophet', ProphetModel(periods=365))
])

In [ ]:
pipeline_prophet

Pipeline(steps=[('prophet', ProphetModel(periods=365))])

In [ ]:
# Fit the pipeline
pipeline_prophet.fit(df)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/1nap14n0.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/y5j7g2e7.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=3702', 'data', 'file=/tmp/tmpeu5819h4/1nap14n0.json', 'init=/tmp/tmpeu5819h4/y5j7g2e7.json', 'output', 'file=/tmp/tmpeu5819h4/prophet_modelmz22tiu9/prophet_model-20240603102933.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:29:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:29:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:n_changepoints greater than number of observations. Using 0.
INFO:prophet:n_changepoints greater than number of observations. Using 4.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/wt4wtsr5.json
DEBUG:cmdstanpy:input tempfile: 

Skipping combination Location: Corporate, ItemName: Vivabond (5Ltr.) due to insufficient data
Skipping combination Location: Corporate, ItemName: Fresmin F 10kg due to insufficient data
Skipping combination Location: Corporate, ItemName: Stable Bleaching Powder 25kg due to insufficient data
Skipping combination Location: Corporate, ItemName: K1 Bio Filter Media due to insufficient data


INFO:prophet:n_changepoints greater than number of observations. Using 0.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/1zwgsx99.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/qb3k6b60.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=8257', 'data', 'file=/tmp/tmpeu5819h4/1zwgsx99.json', 'init=/tmp/tmpeu5819h4/qb3k6b60.json', 'output', 'file=/tmp/tmpeu5819h4/prophet_modelaah31eut/prophet_model-20240603102934.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
10:29:34 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:29:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:n_changepoints greater than number of observations. Using 0.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/5lm1tjl_.json
DEBUG:cmdstanpy:input tempfile:

Skipping combination Location: Karnaphuli, ItemName: Salivet 500ml due to insufficient data
Skipping combination Location: Karnaphuli, ItemName: Salivet 1ltr due to insufficient data
Skipping combination Location: Karnaphuli, ItemName: Viusid Vet 500ml due to insufficient data


DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/rg_cbqet.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=77996', 'data', 'file=/tmp/tmpeu5819h4/sm3xypnq.json', 'init=/tmp/tmpeu5819h4/rg_cbqet.json', 'output', 'file=/tmp/tmpeu5819h4/prophet_modeloo7ijhmj/prophet_model-20240603102935.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:29:35 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:29:35 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/jh8tep92.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/2ve8uirg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['

...

Skipping combination Location: Rupsha, ItemName: Fresmin F 10kg due to insufficient data


10:29:41 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:n_changepoints greater than number of observations. Using 0.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/rszgz8r_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeu5819h4/xqf79ytz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=45813', 'data', 'file=/tmp/tmpeu5819h4/rszgz8r_.json', 'init=/tmp/tmpeu5819h4/xqf79ytz.json', 'output', 'file=/tmp/tmpeu5819h4/prophet_model8ky57n4m/prophet_model-20240603102941.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
10:29:41 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


Skipping combination Location: Tista, ItemName: Fresmin F 10kg due to insufficient data


10:29:41 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Pipeline(steps=[('prophet', ProphetModel(periods=365))])

In [ ]:
forecast= pipeline_prophet.transform(df)

In [ ]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,...,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,Location,ItemName
0,2022-04-23,16000.548400,-8229.537016,44854.655906,16000.548400,16000.548400,1953.617561,1953.617561,1953.617561,-3743.393487,...,-3743.393487,5697.011048,5697.011048,5697.011048,0.0,0.0,0.0,17954.165961,Brahmaputra,Clean 100ml
1,2022-04-25,16005.183471,458.318515,58923.384029,16005.183471,16005.183471,12371.391144,12371.391144,12371.391144,5520.639214,...,5520.639214,6850.751930,6850.751930,6850.751930,0.0,0.0,0.0,28376.574615,Brahmaputra,Clean 100ml
2,2022-04-26,16007.501007,-2465.189910,52963.626146,16007.501007,16007.501007,9564.648227,9564.648227,9564.648227,2046.022149,...,2046.022149,7518.626078,7518.626078,7518.626078,0.0,0.0,0.0,25572.149234,Brahmaputra,Clean 100ml
3,2022-04-27,16009.818543,-4382.462623,53246.059469,16009.818543,16009.818543,9139.882909,9139.882909,9139.882909,924.917142,...,924.917142,8214.965767,8214.965767,8214.965767,0.0,0.0,0.0,25149.701452,Brahmaputra,Clean 100ml
4,2022-07-26,16218.396765,1268.485300,59882.919274,16218.396765,16218.396765,15431.551273,15431.551273,15431.551273,2046.022149,...,2046.022149,13385.529125,13385.529125,13385.529125,0.0,0.0,0.0,31649.948038,Brahmaputra,Clean 100ml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11387,2024-10-17,35449.867629,47833.981575,121310.855987,35449.844645,35449.889160,48755.687027,48755.687027,48755.687027,-2392.179680,...,-2392.179680,51147.866707,51147.866707,51147.866707,0.0,0.0,0.0,84205.554656,Turag,Clean 100ml
11388,2024-10-18,35453.570452,47501.617587,117480.221158,35453.547407,35453.592047,47133.970311,47133.970311,47133.970311,-7461.329998,...,-7461.329998,54595.300310,54595.300310,54595.300310,0.0,0.0,0.0,82587.540764,Turag,Clean 100ml
11389,2024-10-19,35457.273275,39933.118472,111638.063719,35457.250113,35457.294934,42382.802741,42382.802741,42382.802741,-15789.279088,...,-15789.279088,58172.081829,58172.081829,58172.081829,0.0,0.0,0.0,77840.076016,Turag,Clean 100ml
11390,2024-10-20,35460.976098,59145.812271,133540.173248,35460.952816,35460.997822,60768.986280,60768.986280,60768.986280,-1051.260456,...,-1051.260456,61820.246736,61820.246736,61820.246736,0.0,0.0,0.0,96229.962378,Turag,Clean 100ml


In [ ]:
# Aggregate predictions on a weekly basis
weekly_predictions = forecast.resample('W-Mon', on='ds').sum()
weekly_predictions

,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,Location,ItemName
ds,,,,,,,,,,,,,,,,,,,,
2022-04-25,7.157403e+05,-6.662601e+04,1.566541e+06,7.157403e+05,7.157403e+05,3.120782e+04,3.120782e+04,3.120782e+04,17883.467599,17883.467599,17883.467599,1.332435e+04,1.332435e+04,1.332435e+04,0.0,0.0,0.0,7.469481e+05,TuragBrahmaputraGumtiKaratoaKarnaphuliPadmaTur...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
2022-05-02,4.685378e+05,-1.182485e+05,1.043640e+06,4.685378e+05,4.685378e+05,-4.994123e+03,-4.994123e+03,-4.994123e+03,-7415.212787,-7415.212787,-7415.212787,2.421089e+03,2.421089e+03,2.421089e+03,0.0,0.0,0.0,4.635437e+05,BrahmaputraGumtiKirtonkholaKopotakkhoMuhuriPad...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
2022-05-09,2.537853e+05,-1.759314e+05,3.806579e+05,2.537853e+05,2.537853e+05,-1.542471e+05,-1.542471e+05,-1.542471e+05,19833.010302,19833.010302,19833.010302,-1.740801e+05,-1.740801e+05,-1.740801e+05,0.0,0.0,0.0,9.953827e+04,GumtiKopotakkhoPadmaRupshaKaratoaPadmaRupshaSurma,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
2022-05-16,4.987385e+05,-1.849193e+05,6.571331e+05,4.987385e+05,4.987385e+05,-2.648634e+05,-2.648634e+05,-2.648634e+05,23174.772219,23174.772219,23174.772219,-2.880382e+05,-2.880382e+05,-2.880382e+05,0.0,0.0,0.0,2.338751e+05,KarnaphuliKopotakkhoPadmaTistaKopotakkhoPadmaT...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
2022-05-23,2.359561e+05,-8.874064e+04,5.765315e+05,2.359561e+05,2.359561e+05,8.232355e+03,8.232355e+03,8.232355e+03,138317.899056,138317.899056,138317.899056,-1.300855e+05,-1.300855e+05,-1.300855e+05,0.0,0.0,0.0,2.441885e+05,KopotakkhoMuhuriKopotakkhoPadmaMuhuriKaratoaKo...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-30,1.373904e+07,5.479143e+06,1.267563e+07,1.308638e+07,1.446212e+07,-4.699470e+06,-4.699470e+06,-4.699470e+06,-10108.790611,-10108.790611,-10108.790611,-4.689361e+06,-4.689361e+06,-4.689361e+06,0.0,0.0,0.0,9.039573e+06,BrahmaputraCorporateGumtiKaratoaKarnaphuliKopo...,Clean 100mlGentech - P.S (20 LTr.)Clean 100mlC...
2024-10-07,8.072572e+06,1.953629e+06,8.088185e+06,7.784997e+06,8.387214e+06,-3.065666e+06,-3.065666e+06,-3.065666e+06,391251.960643,391251.960643,391251.960643,-3.456918e+06,-3.456918e+06,-3.456918e+06,0.0,0.0,0.0,5.006906e+06,BrahmaputraCorporateGumtiKaratoaKarnaphuliKopo...,Clean 100mlGentech - P.S (20 LTr.)Clean 100mlC...
2024-10-14,4.068515e+06,-3.470897e+05,5.169850e+06,4.068270e+06,4.068757e+06,-1.653824e+06,-1.653824e+06,-1.653824e+06,7861.034984,7861.034984,7861.034984,-1.661685e+06,-1.661685e+06,-1.661685e+06,0.0,0.0,0.0,2.414691e+06,BrahmaputraGumtiKaratoaKopotakkhoMuhuriPadmaRu...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...


In [ ]:
# Aggregate predictions on a monthly basis
monthly_predictions = forecast.resample('M', on='ds').sum()
monthly_predictions

,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,Location,ItemName
ds,,,,,,,,,,,,,,,,,,,,
2022-04-30,1.184278e+06,-1.848745e+05,2.610182e+06,1.184278e+06,1.184278e+06,2.621369e+04,2.621369e+04,2.621369e+04,1.046825e+04,1.046825e+04,1.046825e+04,1.574544e+04,1.574544e+04,1.574544e+04,0.0,0.0,0.0,1.210492e+06,TuragBrahmaputraGumtiKaratoaKarnaphuliPadmaTur...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
2022-05-31,1.187479e+06,-5.813810e+05,1.996492e+06,1.187479e+06,1.187479e+06,-4.840839e+05,-4.840839e+05,-4.840839e+05,1.744287e+05,1.744287e+05,1.744287e+05,-6.585126e+05,-6.585126e+05,-6.585126e+05,0.0,0.0,0.0,7.033948e+05,GumtiKopotakkhoPadmaRupshaKaratoaPadmaRupshaSu...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
2022-06-30,4.702775e+05,-1.687462e+05,9.239383e+05,4.702775e+05,4.702775e+05,-9.537583e+04,-9.537583e+04,-9.537583e+04,-6.175485e+04,-6.175485e+04,-6.175485e+04,-3.362097e+04,-3.362097e+04,-3.362097e+04,0.0,0.0,0.0,3.749017e+05,KopotakkhoKopotakkhoTuragKirtonkholaKaratoaKir...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
2022-07-31,-8.348979e+06,-7.806133e+06,-4.840531e+06,-8.348979e+06,-8.348979e+06,2.022718e+06,2.022718e+06,2.022718e+06,1.997269e+05,1.997269e+05,1.997269e+05,1.822991e+06,1.822991e+06,1.822991e+06,0.0,0.0,0.0,-6.326261e+06,KaratoaKaratoaKaratoaKaratoaKopotakkhoKaratoaK...,Fresmin F 10kgFresmin F 10kgFresmin F 10kgFres...
2022-08-31,-2.100528e+07,-2.661543e+07,-2.026747e+07,-2.100528e+07,-2.100527e+07,-2.432565e+06,-2.432565e+06,-2.432565e+06,-2.327572e+05,-2.327572e+05,-2.327572e+05,-2.199807e+06,-2.199807e+06,-2.199807e+06,0.0,0.0,0.0,-2.343784e+07,GumtiKaratoaMuhuriRupshaTuragBrahmaputraGumtiK...,Clean 100mlFresmin F 10kgViusid Vet 500mlClean...
2022-09-30,-3.369859e+07,-2.619864e+07,-2.105349e+07,-3.369859e+07,-3.369859e+07,1.005835e+07,1.005835e+07,1.005835e+07,4.686548e+05,4.686548e+05,4.686548e+05,9.589691e+06,9.589691e+06,9.589691e+06,0.0,0.0,0.0,-2.364024e+07,KaratoaKaratoaKarnaphuliKirtonkholaKopotakkhoM...,Clean 100mlFresmin F 10kgClean 100mlClean 100m...
2022-10-31,-4.869551e+07,-5.579495e+07,-5.202846e+07,-4.869551e+07,-4.869551e+07,-5.204658e+06,-5.204658e+06,-5.204658e+06,4.348860e+05,4.348860e+05,4.348860e+05,-5.639544e+06,-5.639544e+06,-5.639544e+06,0.0,0.0,0.0,-5.390017e+07,KaratoaMuhuriTuragKaratoaKaratoaKopotakkhoMuhu...,Fresmin F 10kgViusid Vet 500mlClean 100mlClean...
2022-11-30,-6.072807e+07,-7.237198e+07,-6.528070e+07,-6.072808e+07,-6.072807e+07,-8.097586e+06,-8.097586e+06,-8.097586e+06,-2.256594e+05,-2.256594e+05,-2.256594e+05,-7.871927e+06,-7.871927e+06,-7.871927e+06,0.0,0.0,0.0,-6.882566e+07,BrahmaputraGumtiKaratoaKaratoaKopotakkhoMuhuri...,Fresmin F 10kgClean 100mlClean 100mlFresmin F ...
2022-12-31,-7.449534e+07,-6.622838e+07,-5.628369e+07,-7.449535e+07,-7.449534e+07,1.324463e+07,1.324463e+07,1.324463e+07,3.997392e+04,3.997392e+04,3.997392e+04,1.320465e+07,1.320465e+07,1.320465e+07,0.0,0.0,0.0,-6.125071e+07,BrahmaputraKaratoaKirtonkholaKopotakkhoMuhuriB...,Fresmin F 10kgFresmin F 10kgClean 100mlFresmin...


In [ ]:
# Aggregate predictions on a half_yearly basis
half_yearly_predictions = forecast.resample('6M', on='ds').sum()
half_yearly_predictions

,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,Location,ItemName
ds,,,,,,,,,,,,,,,,,,,,
2022-04-30,1.184278e+06,-1.848745e+05,2.610182e+06,1.184278e+06,1.184278e+06,2.621369e+04,2.621369e+04,2.621369e+04,1.046825e+04,1.046825e+04,1.046825e+04,1.574544e+04,1.574544e+04,1.574544e+04,0.0,0.0,0.0,1.210492e+06,TuragBrahmaputraGumtiKaratoaKarnaphuliPadmaTur...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
2022-10-31,-1.100906e+08,-1.171653e+08,-9.526952e+07,-1.100906e+08,-1.100906e+08,3.864381e+06,3.864381e+06,3.864381e+06,9.831844e+05,9.831844e+05,9.831844e+05,2.881196e+06,2.881196e+06,2.881196e+06,0.0,0.0,0.0,-1.062262e+08,GumtiKopotakkhoPadmaRupshaKaratoaPadmaRupshaSu...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
2023-04-30,-5.552538e+08,-5.972569e+08,-5.373896e+08,-5.552538e+08,-5.552537e+08,-1.209451e+07,-1.209451e+07,-1.209451e+07,9.610947e+05,9.610947e+05,9.610947e+05,-1.305561e+07,-1.305561e+07,-1.305561e+07,0.0,0.0,0.0,-5.673483e+08,BrahmaputraGumtiKaratoaKaratoaKopotakkhoMuhuri...,Fresmin F 10kgClean 100mlClean 100mlFresmin F ...
2023-10-31,-4.812326e+08,-5.335890e+08,-4.505879e+08,-4.812340e+08,-4.812302e+08,-1.081879e+07,-1.081879e+07,-1.081879e+07,2.693235e+06,2.693235e+06,2.693235e+06,-1.351203e+07,-1.351203e+07,-1.351203e+07,0.0,0.0,0.0,-4.920514e+08,BrahmaputraKaratoaKaratoaKirtonkholaKopotakkho...,Fresmin F 10kgVivabond (5Ltr.)Fresmin F 10kgCl...
2024-04-30,-1.140399e+08,-1.817456e+08,-1.995889e+06,-1.170176e+08,-1.105657e+08,2.193684e+07,2.193684e+07,2.193684e+07,8.650619e+04,8.650619e+04,8.650619e+04,2.185034e+07,2.185034e+07,2.185034e+07,0.0,0.0,0.0,-9.210311e+07,BrahmaputraCorporateCorporateCorporateCorporat...,Clean 100mlGentech - P.S (20 LTr.)Clean 100mlV...
2024-10-31,3.097674e+08,2.165051e+08,3.918650e+08,2.991608e+08,3.213377e+08,-6.093295e+06,-6.093295e+06,-6.093295e+06,-3.512288e+05,-3.512288e+05,-3.512288e+05,-5.742066e+06,-5.742066e+06,-5.742066e+06,0.0,0.0,0.0,3.036741e+08,BrahmaputraCorporateCorporateCorporateCorporat...,Clean 100mlGentech - P.S (20 LTr.)Clean 100mlV...


In [ ]:
yearly_predictions = forecast.resample('Y', on='ds').sum()
yearly_predictions

,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,Location,ItemName
ds,,,,,,,,,,,,,,,,,,,,
2022-12-31,-2.441297e+08,-2.559505e+08,-2.142237e+08,-2.441297e+08,-2.441297e+08,9.037636e+06,9.037636e+06,9.037636e+06,8.079672e+05,8.079672e+05,8.079672e+05,8.229669e+06,8.229669e+06,8.229669e+06,0.0,0.0,0.0,-2.350921e+08,TuragBrahmaputraGumtiKaratoaKarnaphuliPadmaTur...,Clean 100mlClean 100mlClean 100mlClean 100mlCl...
2023-12-31,-9.525462e+08,-9.311937e+08,-7.463921e+08,-9.527611e+08,-9.522753e+08,1.136885e+08,1.136885e+08,1.136885e+08,1.634300e+06,1.634300e+06,1.634300e+06,1.120542e+08,1.120542e+08,1.120542e+08,0.0,0.0,0.0,-8.388576e+08,BrahmaputraGumtiKaratoaKirtonkholaKopotakkhoKo...,Fresmin F 10kgClean 100mlFresmin F 10kgClean 1...
2024-12-31,2.470107e+08,-2.629240e+07,2.698480e+08,2.336399e+08,2.617868e+08,-1.259053e+08,-1.259053e+08,-1.259053e+08,1.940992e+06,1.940992e+06,1.940992e+06,-1.278463e+08,-1.278463e+08,-1.278463e+08,0.0,0.0,0.0,1.211053e+08,BrahmaputraCorporateCorporateCorporateCorporat...,Clean 100mlGentech - P.S (20 LTr.)Clean 100mlV...
